In [1]:
import requests
import pprint
import json
import mysql.connector

state_code = "FL"
url_hospitals_per_state = f"http://www.communitybenefitinsight.org/api/get_hospitals.php?state={state_code}"

payload = {}
headers = {
    'Cookie': 'PHPSESSID=nb9s8neh28dr9uuol0eg517l2m'
}

#hospitals for florida
response_hospitals_per_state  = requests.request("GET", url_hospitals_per_state, headers=headers, data=payload)

# Save the JSON object to a file
with open(f"hospital_data_per_state_{state_code}.json", "w") as f:
    json.dump(json.loads(response_hospitals_per_state.content), f, indent=4)

p = pprint.PrettyPrinter(indent=4)

data_state_hosp = json.loads(response_hospitals_per_state.content)
p.pprint(json.loads(response_hospitals_per_state.content))

[   {   'children_hospital_f': 'N',
        'chrch_affl_f': 'N',
        'city': 'Jacksonville',
        'county': 'Duval County',
        'ein': '592142859',
        'fips_state_and_county_code': '12031',
        'hospital_bed_count': '695',
        'hospital_bed_size': '>299 beds',
        'hospital_id': '508',
        'hospital_org_id': '332',
        'medicare_provider_number': '100001',
        'memb_counc_teach_hosps_f': 'Y',
        'name': 'Shands Jacksonville Medical Center',
        'name_cr': 'Shands Jacksonville Medical Center',
        'state': 'FL',
        'street_address': '655 West 8th Street',
        'updated_dt': 'July 11, 2023',
        'urban_location_f': 'Y',
        'zip_code': '32209'},
    {   'children_hospital_f': 'N',
        'chrch_affl_f': 'N',
        'city': 'Boynton Beach',
        'county': 'Palm Beach County',
        'ein': '592447554',
        'fips_state_and_county_code': '12099',
        'hospital_bed_count': '481',
        'hospital_bed_size': '

Get hospitals per ID

In [2]:
hospital_id = "2164"

url_hospital_data_per_id = f"http://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id={hospital_id}"

responce_hospital_data_per_id = requests.request("GET", url_hospital_data_per_id, headers=headers, data=payload)

# Save the JSON object to a file
with open(f"hospital_data_per_id_{hospital_id}.json", "w") as f:
    json.dump(json.loads(responce_hospital_data_per_id.content), f, indent=4)

#p.pprint(json.loads(responce_hospital_data_per_id.content))

Function for calculations


In [3]:
def calculate_and_print_bed_stats(table_name, mycursor,region_type):
    """
    The function will calculate bed count depending on rural or urban region type
    and prints the results.
    For rural region_type = N, and for urban region_type = Y
    """

    calculations = {}
    region_type = region_type.upper()

    if region_type not in ['Y', 'N']:
        raise ValueError("Invalid region type. Must be 'Y' for urban or 'N' for rural.")

    # Total beds
    mycursor.execute(f"SELECT SUM(hospital_bed_count) FROM {table_name} WHERE urban_location_f = '{region_type}'")
    calculations['Total beds count'] = float(mycursor.fetchone()[0])

    # Average beds
    mycursor.execute(f"SELECT AVG(hospital_bed_count) FROM {table_name} WHERE urban_location_f = '{region_type}'")
    calculations['Average'] = float(mycursor.fetchone()[0])

    # Min beds
    mycursor.execute(f"SELECT MIN(hospital_bed_count) FROM {table_name} WHERE urban_location_f = '{region_type}'")
    calculations['Min'] = float(mycursor.fetchone()[0])

    # Max beds
    mycursor.execute(f"SELECT MAX(hospital_bed_count) FROM {table_name} WHERE urban_location_f = '{region_type}'")
    calculations['Max'] = float(mycursor.fetchone()[0])

    # Pretty-print the results
    pp = pprint.PrettyPrinter()
    if region_type == 'Y': 
        print("Statistics for Urban Region:")
        pp.pprint(calculations)
    else:
        print("Statistics for Rural Region:")
        pp.pprint(calculations)
    




Function for insertion of the data

In [4]:
def insert_hospital_data(cursor, table_name, hospital_data):
    """
    Inserts hospital data into the specified table.
    """
    try:
        columns = hospital_data.keys()
        values = list(hospital_data.values())
        
        # Construct the INSERT statement
        insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(values))})"
        
        # Execute the query with the values
        cursor.execute(insert_query, values)
        return True

    except mysql.connector.Error as e:
        print(f"MySQL Error: {e}")
        print(f"Error Code: {e.errno}")
        print(f"SQL State: {e.sqlstate}")
        return False
   


Function for error handling of commits

In [5]:
def commit_changes(connection):
    """
    Commits changes to the database and handles exceptions.
    """
    try:
        connection.commit()
        print("Transaction committed successfully.")
    except mysql.connector.Error as e:
        print(f"Error committing transaction: {e}")
        connection.rollback()  # Rollback in case of error


In [6]:
#connect to mysql with env. file

from dotenv import load_dotenv
import os
import mysql.connector
from mysql.connector import Error

load_dotenv('personal_data.env')
db_name = "mydb2"

try: 
    connection = mysql.connector.connect(
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD')
    )

    mycursor = connection.cursor()
    #creating a new database instead of connecting to the existing one
    mycursor.execute(f"create database IF NOT EXISTS {db_name}")
    print(f"Database {db_name} created successfully.")

except Error as e:
    print(f"Error: {e}")


Database mydb2 created successfully.


In [7]:
#selecting created database

mycursor.execute(f"USE {db_name}")

#dynamically create the table based on the first hospital keys


if data_state_hosp:
    first_hosp = data_state_hosp[0]
    column_definitions = []
    for column_name in first_hosp.keys():
        #WE ARE ASSUMING ALL DATA IS IN STRING FORMAT FOR SIMPLICITY
        column_definitions.append(f"{column_name} VARCHAR(255)")



    #creating a table
    table_name = "stateHospitals"
    #dropping the table so data wouldn't double every time script is run
    mycursor.execute(f"drop table if exists {table_name} ")
    mycursor.execute(f"create table if not exists {table_name} ({' ,'.join(column_definitions)})")

#insert all state data into the table
for hospital in data_state_hosp:
    success = insert_hospital_data(mycursor,table_name, hospital)
    if not success:
         print(f"Failed to insert data for hospital: {hospital['name']}")

commit_changes(connection)

Transaction committed successfully.


In [8]:
#change data type of bad count column for further calculations

#ALTER TABLE table_name    
#MODIFY column_name datatype;  
mycursor.execute(f"alter table {table_name} modify hospital_bed_count INT")
commit_changes(connection)

Transaction committed successfully.


In [9]:

#create dictionary to save calculations for both types of regions
rural_calculations = {'Total beds count': None, "Average": None , "Min": None, "Max":None }
urban_calculations = {'Total beds count': None, "Average": None , "Min": None, "Max":None }

#calculate initial stats before adding more data

calculate_and_print_bed_stats(table_name, mycursor, 'N')
calculate_and_print_bed_stats(table_name, mycursor, 'Y')

Statistics for Rural Region:
{'Average': 24.5714, 'Max': 48.0, 'Min': 4.0, 'Total beds count': 172.0}
Statistics for Urban Region:
{'Average': 386.6163, 'Max': 3060.0, 'Min': 19.0, 'Total beds count': 33249.0}


In [10]:
#add some data in the table 
new_hospitals = [
    {
        'hospital_id': '999',
        'hospital_org_id': '999',
        'ein': '999999999',
        'name': 'Improv Health Center',
        'name_cr': 'Improv Health Center',
        'street_address': '123 Health St',
        'city': 'Tampa',
        'state': 'FL',
        'zip_code': '12345',
        'fips_state_and_county_code': '12345',
        'hospital_bed_count': '150',
        'chrch_affl_f': 'N',
        'urban_location_f': 'Y',
        'children_hospital_f': 'N',
        'memb_counc_teach_hosps_f': 'N',
        'medicare_provider_number': '999999',
        'county': 'Health County',
        'hospital_bed_size': '100-299 beds',
        'updated_dt': '2023-11-03'
    },
    {
        'hospital_id': '1000',
        'hospital_org_id': '1000',
        'ein': '888888888',
        'name': 'Carewell Hospital',
        'name_cr': 'Carewell Hospital',
        'street_address': '456 Wellness Blvd',
        'city': 'Miami',
        'state': 'FL',
        'zip_code': '67890',
        'fips_state_and_county_code': '67890',
        'hospital_bed_count': '300',
        'chrch_affl_f': 'Y',
        'urban_location_f': 'N',
        'children_hospital_f': 'Y',
        'memb_counc_teach_hosps_f': 'Y',
        'medicare_provider_number': '888888',
        'county': 'Care County',
        'hospital_bed_size': '>299 beds',
        'updated_dt': '2023-11-03'
    }
]

# Insert the new hospitals into the table
for hospital in new_hospitals:
    success = insert_hospital_data(mycursor, table_name, hospital)
    if not success:
        print(f"Failed to insert new data")
# Commit the changes
commit_changes(connection)

Transaction committed successfully.


In [11]:
#calculate stats for updated table

calculate_and_print_bed_stats(table_name, mycursor, 'N')
calculate_and_print_bed_stats(table_name, mycursor, 'Y')

Statistics for Rural Region:
{'Average': 59.0, 'Max': 300.0, 'Min': 4.0, 'Total beds count': 472.0}
Statistics for Urban Region:
{'Average': 383.8966, 'Max': 3060.0, 'Min': 19.0, 'Total beds count': 33399.0}


In [12]:
#delete added data

mycursor.execute(f"delete from {table_name} where hospital_id = '999' or hospital_id='1000'")
commit_changes(connection)

Transaction committed successfully.


In [13]:
connection.close()
print("Database connection closed.")

Database connection closed.
